In [3]:
# Importing necessary libraries
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('properties.csv')

# dropping irrelevant columns
df = df.drop(['Description', 'Link'], axis=1)

df = df.drop(['Car Space', 'Condominum Name'], axis=1)

# 1. Identify the row(s) with missing coordinates
missing_coords_df = df[df['Longitude'].isnull() | df['Latitude'].isnull()]

# 2. Check the location name of the missing row(s)
# The output of this will help manually find the correct coordinates
print("Location(s) with missing coordinates:")
print(missing_coords_df[['Location', 'Longitude', 'Latitude']])



Location(s) with missing coordinates:
             Location  Longitude  Latitude
337  Songculan, Dauis        NaN       NaN


In [4]:
# 3. manually fill in the missing coordinates based on external lookup since only 1 row is missing and latitude and longitude needs to be accurate
df.loc[df['Location'] == 'Songculan, Dauis', 'Longitude'] = 123.8294
df.loc[df['Location'] == 'Songculan, Dauis', 'Latitude'] = 9.6291

df.loc[df['Location'] == 'Songculan, Dauis', ['Location', 'Longitude', 'Latitude']]

,Location,Longitude,Latitude
337,"Songculan, Dauis",123.8294,9.6291


In [6]:
from sklearn.cluster import KMeans

# Drop 'Location' if Longitude/Latitude are present
df = df.drop('Location', axis=1)

# Cluster by geographic proximity
kmeans = KMeans(n_clusters=10, random_state=42)
df['Location_Cluster'] = kmeans.fit_predict(df[['Longitude', 'Latitude']])

df.head()

,Category,Sub-Category,Land Size(M2),Building Size(M2),Bedrooms,Bathrooms,Longitude,Latitude,Price(Php),Location_Cluster
0,Condominium,"Condominium,3-bedroom",NaN,128.0,3.0,2.0,121.055300,14.537120,19000000.0,0
1,House,"House,single-family-house",101.0,60.0,3.0,2.0,120.665111,15.169115,3700000.0,7
2,House,"House,single-family-house",100.0,75.0,3.0,2.0,125.534347,7.090939,3790000.0,3
3,Land,"Land,agricutural-lot",6132.0,NaN,NaN,NaN,120.470266,17.430680,3066000.0,5
4,Land,"Land,beach-lot",537.0,NaN,NaN,NaN,121.379905,13.691466,20191200.0,0


In [ ]:
# Create a larger, higher quality figure
plt.figure(figsize=(14, 10))

# Enhanced scatter plot with better styling
scatter = plt.scatter(df['Longitude'], df['Latitude'], 
                     c=df['Location_Cluster'], 
                     cmap='tab10',  # Better color palette for distinct clusters
                     s=100,  # Larger points
                     alpha=0.7,  # Slight transparency
                     edgecolors='black',  # Black borders for better visibility
                     linewidth=0.5)

# Plot cluster centers with enhanced styling
plt.scatter(kmeans.cluster_centers_[:, 0], 
           kmeans.cluster_centers_[:, 1], 
           c='red', 
           s=500,  # Larger centers
           marker='*',  # Star marker for centers
           edgecolors='darkred',
           linewidth=2,
           label='Cluster Centers',
           zorder=5)  # Ensure centers appear on top

# Enhanced title and labels
plt.title('Geographic Clustering of Property Locations', 
         fontsize=18, 
         fontweight='bold',
         pad=20)
plt.xlabel('Longitude', fontsize=14, fontweight='bold')
plt.ylabel('Latitude', fontsize=14, fontweight='bold')

# Add colorbar to show cluster assignments
cbar = plt.colorbar(scatter, label='Cluster ID')
cbar.set_label('Cluster ID', fontsize=12, fontweight='bold')

# Enhanced legend
plt.legend(fontsize=12, 
          loc='upper left',
          frameon=True,
          shadow=True,
          fancybox=True)

# Improved grid
plt.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)

# Add cluster statistics as text annotation
n_clusters = len(kmeans.cluster_centers_)
cluster_counts = df['Location_Cluster'].value_counts().sort_index()

stats_text = f"Total Properties: {len(df)}\nNumber of Clusters: {n_clusters}"
plt.text(0.02, 0.98, stats_text,
        transform=plt.gca().transAxes,
        fontsize=11,
        verticalalignment='top',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

# Adjust layout to prevent label cutoff
plt.tight_layout()

plt.show()

# Optional: Print cluster statistics
print("\n=== Cluster Statistics ===")
print(f"Total properties analyzed: {len(df)}")
print(f"\nProperties per cluster:")
for cluster_id in sorted(df['Location_Cluster'].unique()):
    count = len(df[df['Location_Cluster'] == cluster_id])
    percentage = (count / len(df)) * 100
    print(f"  Cluster {cluster_id}: {count} properties ({percentage:.1f}%)")

print(f"\nCluster Centers (Longitude, Latitude):")
for i, center in enumerate(kmeans.cluster_centers_):
    print(f"  Cluster {i}: ({center[0]:.2f}, {center[1]:.2f})")

In [5]:
df.describe()

,Land Size(M2),Building Size(M2),Bedrooms,Bathrooms,Longitude,Latitude,Price(Php)
count,2.131000e+03,2292.000000,2080.000000,2030.000000,3000.000000,3000.000000,3.000000e+03
mean,2.592443e+05,359.700777,3.208413,2.983990,121.182946,14.160380,6.137594e+07
std,1.152421e+07,709.082810,2.323407,2.370004,2.126320,1.387860,2.002761e+08
min,0.000000e+00,0.000000,0.000000,0.000000,15.113011,6.159605,1.000000e+00
25%,1.205000e+02,60.000000,2.000000,1.000000,121.000352,14.357185,6.330976e+06
50%,2.640000e+02,140.000000,3.000000,3.000000,121.037608,14.540867,1.535400e+07
75%,5.585000e+02,330.000000,4.000000,4.000000,121.078310,14.592420,4.800000e+07
max,5.318331e+08,8746.000000,40.000000,35.000000,126.862440,18.275275,8.269039e+09


In [ ]:

# Load the dataset
df_for_chart = pd.read_csv('properties.csv')

# Group by number of bedrooms and compute statistics
bedroom_stats = df_for_chart.groupby('Bedrooms')['Price(Php)'].agg([
    ('Average Price (Php)', 'mean'),
    ('Median Price (Php)', 'median'),
    ('Count', 'count'),
    ('Std Dev', 'std')
]).reset_index()

bedroom_stats.columns = ['Number of Bedrooms', 'Average Price (Php)', 
                         'Median Price (Php)', 'Property Count', 'Std Dev']

# Format prices for better readability
bedroom_stats['Avg Price (Formatted)'] = bedroom_stats['Average Price (Php)'].apply(
    lambda x: f'₱{x:,.0f}'
)
bedroom_stats['Median Price (Formatted)'] = bedroom_stats['Median Price (Php)'].apply(
    lambda x: f'₱{x:,.0f}'
)

# Display the detailed statistics table
print("=" * 90)
print("PRICE ANALYSIS BY NUMBER OF BEDROOMS")
print("=" * 90)
print(bedroom_stats[['Number of Bedrooms', 'Avg Price (Formatted)', 
                     'Median Price (Formatted)', 'Property Count']].to_string(index=False))
print("=" * 90)

# Create enhanced visualization with subplots
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Comprehensive Price Analysis by Number of Bedrooms', 
             fontsize=18, fontweight='bold', y=0.995)

# Plot 1: Average Price Line Chart
ax1 = axes[0, 0]
ax1.plot(bedroom_stats['Number of Bedrooms'], 
         bedroom_stats['Average Price (Php)'], 
         marker='o', 
         linewidth=2.5, 
         markersize=10,
         color='#2E86AB',
         markerfacecolor='#F24236',
         markeredgecolor='darkred',
         markeredgewidth=2)

# Add value labels on points
for idx, row in bedroom_stats.iterrows():
    ax1.annotate(f"₱{row['Average Price (Php)']/1e6:.1f}M",
                xy=(row['Number of Bedrooms'], row['Average Price (Php)']),
                xytext=(0, 10),
                textcoords='offset points',
                ha='center',
                fontsize=9,
                fontweight='bold',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7))

ax1.set_title('Average Price Trend', fontsize=14, fontweight='bold', pad=10)
ax1.set_xlabel('Number of Bedrooms', fontsize=12, fontweight='bold')
ax1.set_ylabel('Average Price (Php)', fontsize=12, fontweight='bold')
ax1.grid(True, alpha=0.3, linestyle='--')
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'₱{x/1e6:.1f}M'))

# Plot 2: Bar Chart with Property Count
ax2 = axes[0, 1]
bars = ax2.bar(bedroom_stats['Number of Bedrooms'], 
               bedroom_stats['Average Price (Php)'],
               color='#A23B72',
               alpha=0.7,
               edgecolor='black',
               linewidth=1.5)

# Add property count labels on bars
for idx, (bar, row) in enumerate(zip(bars, bedroom_stats.itertuples())):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
            f"n={row._4}",  # Property Count
            ha='center', va='bottom',
            fontsize=9, fontweight='bold')

ax2.set_title('Average Price Distribution', fontsize=14, fontweight='bold', pad=10)
ax2.set_xlabel('Number of Bedrooms', fontsize=12, fontweight='bold')
ax2.set_ylabel('Average Price (Php)', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3, linestyle='--', axis='y')
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'₱{x/1e6:.1f}M'))

# Plot 3: Average vs Median Comparison
ax3 = axes[1, 0]
x_pos = np.arange(len(bedroom_stats))
width = 0.35

bars1 = ax3.bar(x_pos - width/2, bedroom_stats['Average Price (Php)'], 
                width, label='Average', color='#4ECDC4', 
                edgecolor='black', linewidth=1)
bars2 = ax3.bar(x_pos + width/2, bedroom_stats['Median Price (Php)'], 
                width, label='Median', color='#FF6B6B',
                edgecolor='black', linewidth=1)

ax3.set_title('Average vs Median Price Comparison', fontsize=14, fontweight='bold', pad=10)
ax3.set_xlabel('Number of Bedrooms', fontsize=12, fontweight='bold')
ax3.set_ylabel('Price (Php)', fontsize=12, fontweight='bold')
ax3.set_xticks(x_pos)
ax3.set_xticklabels(bedroom_stats['Number of Bedrooms'])
ax3.legend(fontsize=11, loc='upper left')
ax3.grid(True, alpha=0.3, linestyle='--', axis='y')
ax3.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'₱{x/1e6:.1f}M'))

# Plot 4: Property Count by Bedrooms
ax4 = axes[1, 1]
colors = plt.cm.viridis(np.linspace(0.3, 0.9, len(bedroom_stats)))
bars = ax4.bar(bedroom_stats['Number of Bedrooms'], 
               bedroom_stats['Property Count'],
               color=colors,
               edgecolor='black',
               linewidth=1.5)

# Add count labels
for bar in bars:
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}',
            ha='center', va='bottom',
            fontsize=11, fontweight='bold')

ax4.set_title('Number of Properties by Bedrooms', fontsize=14, fontweight='bold', pad=10)
ax4.set_xlabel('Number of Bedrooms', fontsize=12, fontweight='bold')
ax4.set_ylabel('Number of Properties', fontsize=12, fontweight='bold')
ax4.grid(True, alpha=0.3, linestyle='--', axis='y')

plt.tight_layout()
plt.show()

# Print additional insights
print("\n" + "=" * 90)
print("KEY INSIGHTS")
print("=" * 90)

# Price increase per bedroom
price_increases = bedroom_stats['Average Price (Php)'].diff()
for i in range(1, len(bedroom_stats)):
    prev_bed = bedroom_stats.iloc[i-1]['Number of Bedrooms']
    curr_bed = bedroom_stats.iloc[i]['Number of Bedrooms']
    increase = price_increases.iloc[i]
    pct_increase = (increase / bedroom_stats.iloc[i-1]['Average Price (Php)']) * 100
    print(f"• {prev_bed} → {curr_bed} bedrooms: +₱{increase:,.0f} (+{pct_increase:.1f}%)")

print(f"\n• Most common: {bedroom_stats.loc[bedroom_stats['Property Count'].idxmax(), 'Number of Bedrooms']:.0f} bedrooms "
      f"({bedroom_stats['Property Count'].max()} properties)")
print(f"• Highest average price: {bedroom_stats.loc[bedroom_stats['Average Price (Php)'].idxmax(), 'Number of Bedrooms']:.0f} bedrooms "
      f"(₱{bedroom_stats['Average Price (Php)'].max():,.0f})")
print("=" * 90)